In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import pandas as pd
import numpy as np
import csv


from sklearn.utils import shuffle

import pandas as pd
from PIL import Image
import numpy as np
import os
import csv
import re
from sklearn.model_selection import StratifiedKFold

In [2]:
np.random.seed(1024)
pos_data = pd.read_csv(r'sentence_embedding_list.csv', header=None)
neg_data = pd.read_csv(r'sentence_embedding_list_negative.csv', header=None)
com_data = pos_data.append(neg_data)
raw_data = shuffle(com_data)
raw_data.to_csv('suffle.csv', encoding = 'utf-8', header = None, index = False)
labels_dataset = raw_data[0]
labels_dataset = labels_dataset.tolist()
raw_data

,0,1
25,1,[-2.89027214e-01 -4.38273013e-01 -3.22299331e-...
89,0,[-7.12146819e-01 -4.24583733e-01 -1.29917473e-...
597,1,[-2.12954760e-01 -8.21731985e-02 -2.26242959e-...
153,1,[-5.63706942e-02 -3.41074973e-01 -1.33537129e-...
711,0,[-4.71603841e-01 -6.57357454e-01 1.23730779e-...
...,...,...
601,1,[ 1.25624493e-01 -1.17772870e-01 2.84849796e-...
613,1,[-2.85503119e-01 -2.00217754e-01 -7.87503868e-...
492,1,[-1.12773709e-01 -4.28319186e-01 -1.97886527e-...
609,1,[-1.75341040e-01 8.72757733e-02 -1.16240516e-...


In [3]:
raw = pd.read_csv(r'suffle.csv', header=None)
dataset = []
for data in raw[1].tolist() :
    num = re.findall('[0-9e.-0-9+]+',data)
    np_num = np.array([float(i) for i in num])
    dataset.append(np_num) 

In [4]:
batch_size = 32  #一個撮（批次）的大小

In [5]:
# 首先，我們定義索引陣列indices，它相當於對所有dataset中數據的編碼
# 然後定義索引indices_train表示測試級的索引，indices_val來表示校驗集數據的索引，indices_test表示測試集的索引
indices = range(len(dataset))
data_size = len(dataset) // 10
indices_train = indices[2 * data_size :]
indices_val = indices[: data_size]
indices_test = indices[data_size : 2 * data_size]

# 根據這些索引，構造三個數據集的SubsetRandomSampler採樣器，它會對索引進行採樣
sampler_train = torch.utils.data.sampler.SubsetRandomSampler(indices_train)
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
sampler_test = torch.utils.data.sampler.SubsetRandomSampler(indices_test)

# 根據三個採樣器來定義加載器，注意將sampler_train、sampler_val和sampler_test分別賦值給了train_loader、validation_loader和test_loader
#注意:labels集只需用相同定義的sampler就可以採樣

train_loader_train = torch.utils.data.DataLoader(dataset= dataset, 
                                           batch_size= batch_size, 
                                           sampler = sampler_train)
train_loader_labels = torch.utils.data.DataLoader(dataset= labels_dataset, 
                                           batch_size= batch_size, 
                                           sampler = sampler_train)

validation_loader_test = torch.utils.data.DataLoader(dataset = dataset,
                                                batch_size = batch_size,
                                                sampler = sampler_val)
validation_loader_labels = torch.utils.data.DataLoader(dataset =labels_dataset,
                                                batch_size = batch_size,
                                                sampler = sampler_val)

test_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=batch_size, 
                                          sampler = sampler_test)
test_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=batch_size, 
                                          sampler = sampler_test)




In [6]:
len(sampler_train)

1442

In [17]:
num_classes = 2  #標簽的種類數
num_epochs = 100  #訓練的總循環周期

size = 768
hidden1 = 100
hidden2 = 100
out = 2

In [18]:
class NN(nn.Module):
    def __init__(self, size, hidden1, hidden2, num_classes):
        # 該函數在創建一個NN對象的時候，即調用如下語句：model=NN()，就會被調用
        # 首先調用父類相應的構造函數
        super(NN, self).__init__()
        
        # 其次構造NN需要用到的各個神經模塊。
        '''注意，定義組件並沒有真正搭建這些組件，只是把基本建築磚塊先找好'''
        
        # 全連線層
        self.fc1 = nn.Linear(size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, num_classes) 
        
        #BN
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.bn2 = nn.BatchNorm1d(hidden2)
        
        
        
        
        

    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(self.bn1(x))
        x = F.dropout(x, training  = self.training)
        
        x = self.fc2(x)
        x = F.relu(self.bn2(x))
        x = F.dropout(x, training  = self.training)
        
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)#輸出層為log_softmax，即概率對數值log(p(x))。采用log_softmax可以使得後面的交叉熵計算更快

        return x

In [19]:
model = NN(size, hidden1, hidden2, num_classes)
print(model)

NN(
  (fc1): Linear(in_features=768, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=2, bias=True)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [30]:
params = list(model.parameters())
print('引數: ', params)
# conv1.weight
print('第一個引數大小: ', params[-1].size())

引數:  [Parameter containing:
tensor([[ 0.0364,  0.0138, -0.0203,  ..., -0.0188,  0.0357,  0.0220],
        [-0.0176,  0.0055, -0.0191,  ...,  0.0167, -0.0014,  0.0052],
        [-0.0213,  0.0102, -0.0222,  ...,  0.0331,  0.0354, -0.0058],
        ...,
        [ 0.0042,  0.0344,  0.0148,  ..., -0.0129, -0.0369, -0.0166],
        [-0.0022, -0.0056,  0.0042,  ...,  0.0232, -0.0127, -0.0052],
        [ 0.0414,  0.0192,  0.0142,  ..., -0.0173,  0.0044, -0.0210]],
       requires_grad=True), Parameter containing:
tensor([-0.0174,  0.0209, -0.0394,  0.0160,  0.0142, -0.0124,  0.0214,  0.0087,
         0.0392, -0.0297, -0.0267, -0.0166, -0.0292, -0.0350, -0.0484,  0.0008,
         0.0238,  0.0117,  0.0141,  0.0168,  0.0160,  0.0066, -0.0281,  0.0384,
        -0.0416, -0.0042, -0.0056, -0.0209, -0.0304, -0.0068, -0.0330,  0.0014,
         0.0193, -0.0052,  0.0077, -0.0200, -0.0314, -0.0010, -0.0216,  0.0175,
        -0.0175, -0.0461,  0.0251, -0.0082, -0.0385,  0.0332, -0.0264, -0.0135,
        

In [21]:
model = NN(size, hidden1, hidden2, num_classes)

# 損失函數為交叉熵
criterion = nn.CrossEntropyLoss()
# 優化算法為Adam，可以自動調節學習率
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)



def rightness(predictions, labels):
    """计算预测错误率的函数，其中predictions是模型给出的一组预测结果，batch_size行num_classes列的矩阵，labels是数据之中的正确答案"""
    pred = torch.max(predictions.data, 1)[1] # 对于任意一行（一个样本）的输出值的第1个维度，求最大，得到每一行的最大元素的下标
    rights = pred.eq(labels.data.view_as(pred)).sum() #将下标与labels中包含的类别进行比较，并累计得到比较正确的数量
    return rights, len(labels) #返回正确的数量和这一次一共比较了多少元素

In [22]:
for i, data in enumerate(zip(train_loader_train, train_loader_labels)):
    x, y = data
    z = torch.tensor(x, requires_grad = True, dtype = torch.float) 
    
model(z)

tensor([[-1.0721, -0.4190],
        [-0.4938, -0.9423]], grad_fn=<LogSoftmaxBackward>)

In [23]:
for i, data in enumerate(zip(train_loader_train, train_loader_labels)):
    x, y = data
    print(x, y)

tensor([[-0.3938, -0.3830, -0.0389,  ..., -0.0270,  0.3251,  0.4885],
        [ 0.0475, -0.2420, -0.4028,  ..., -0.2092,  0.0156,  0.4131],
        [-0.3850, -0.4682,  0.2535,  ..., -0.0760,  0.1846,  0.2591],
        ...,
        [-0.3633, -0.7579, -0.3822,  ...,  0.4130, -0.1240,  0.0885],
        [-0.2959, -0.1518, -0.1805,  ..., -0.0308, -0.2556,  0.5253],
        [ 0.0984, -0.1236,  0.0217,  ...,  0.1759, -0.0761,  0.6120]],
       dtype=torch.float64) tensor([0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 1])
tensor([[-0.2502, -0.3550, -0.2655,  ..., -0.4906,  0.2951,  0.1412],
        [ 0.1325, -0.2030, -0.3874,  ...,  0.4411, -0.2838,  0.4935],
        [-0.2836, -0.2044, -0.2565,  ..., -0.1048,  0.0946,  0.2902],
        ...,
        [-0.4115,  0.1867, -0.6660,  ..., -0.2391,  0.2860,  0.7592],
        [-0.3012, -0.2645, -0.1858,  ..., -0.1388, -0.2219,  0.4377],
        [ 0.0087, -0.5075, -0.0534,  ...,  0.1153,  0.2975, -0

In [24]:
import warnings
warnings.filterwarnings('ignore')
record = []
losses = []
records = []
for epoch in range(num_epochs):
    train_losses = []
    train_rights = []
    model.train()
    for batch_idx, train in enumerate(zip(train_loader_train, train_loader_labels)):
        x, y = train
        
        
        x = torch.tensor(x, requires_grad = True, dtype = torch.float)   # x 最後會被轉成 [0-1] 之間的數

        y = torch.tensor(np.array([z for z in y ]), dtype = torch.long)  # 所以 y 必須是llong 而不是
        
        #np.array([z for z in y ]) 因為np.array只能轉一個值，若要list裡面全部的值，需改成這樣
        
        # 清空梯度
        optimizer.zero_grad()
        # 模型預測
        predict = model(x)
        # 計算損失函數
        loss = criterion(predict, y)
        # 將損失函數數值加入到列表中
        train_losses.append(loss.data.numpy())
        # 開始進行梯度反傳
        loss.backward()
        # 開始對參數進行一步優化
        optimizer.step()
        #計算準確率所需數值，返回數值為（正確樣例數，總樣本數）
        right = rightness(predict, y)
        #將計算結果裝到列表容器train_rights中
        train_rights.append(right)
        
        
        
        
        
        # 每隔200步，跑一下校驗數據集的數據，輸出臨時結果
        if batch_idx % 10 == 0:
        
            losses.append(np.mean(train_losses))
            model.eval()
            val_losses = []
            val_rights = []
            # 在所有校驗數據集上實驗
            with torch.no_grad():
                
                for j, val in enumerate(zip(validation_loader_test, validation_loader_labels)):
                    x, y = val
                    x = torch.tensor(x, requires_grad = True, dtype = torch.float)
                    y = torch.tensor(np.array([z for z in y ]), dtype = torch.long)
                    predict = model(x)
                    # 調用rightness函數計算準確度
                    right = rightness(predict, y)
                    val_rights.append(right)
                    loss = criterion(predict, y)
                    val_losses.append(loss.data.numpy())



                # 分別計算在目前已經計算過的測試數據集，以及全部校驗集上模型的表現：分類準確率
                #train_r為一個二元組，分別記錄目前已經經歷過的所有訓練集中分類正確的數量和該集合中總的樣本數，
                #train_r[0]/train_r[1]就是訓練集的分類準確度，同樣，val_r[0]/val_r[1]就是校驗集上的分類準確度
                train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
                #val_r為一個二元組，分別記錄校驗集中分類正確的數量和該集合中總的樣本數
                val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
                #打印準確率等數值，其中正確率為本訓練周期Epoch開始後到目前撮的正確率的平均值
                print(val_r)
                print('訓練周期: {} [{}/{} ({:.0f}%)]\tTotal Loss: {:.2f}\ttraining Loss: {:.2f}\tval Loss: {:.2f}\t訓練正確率: {:.2f}%\t校驗正確率: {:.2f}%'.format(
                    epoch, batch_idx * batch_size, len(dataset),
                    100. * batch_idx / len(train_loader_train), 
                    np.mean(losses),
                    np.mean(train_losses),
                    np.mean(val_losses),
                    100. * train_r[0].numpy() / train_r[1], 
                    100. * val_r[0].numpy() / val_r[1]))
                #將準確率和權重等數值加載到容器中，以方便後續處理
                record.append((100 - 100. * train_r[0] / train_r[1], 100 - 100. * val_r[0] / val_r[1]))
                records.append([np.mean(train_losses), np.mean(val_losses), train_r, val_r, losses])

(tensor(92), 180)
訓練周期: 0 [0/1802 (0%)]	Total Loss: 0.73	training Loss: 0.73	val Loss: 0.69	訓練正確率: 43.75%	校驗正確率: 51.11%
(tensor(92), 180)
訓練周期: 0 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.43%	校驗正確率: 51.11%
(tensor(90), 180)
訓練周期: 0 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.11%	校驗正確率: 50.00%
(tensor(90), 180)
訓練周期: 0 [960/1802 (65%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.69%	校驗正確率: 50.00%
(tensor(88), 180)
訓練周期: 0 [1280/1802 (87%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.17%	校驗正確率: 48.89%
(tensor(92), 180)
訓練周期: 1 [0/1802 (0%)]	Total Loss: 0.73	training Loss: 0.87	val Loss: 0.69	訓練正確率: 46.88%	校驗正確率: 51.11%
(tensor(86), 180)
訓練周期: 1 [320/1802 (22%)]	Total Loss: 0.73	training Loss: 0.71	val Loss: 0.69	訓練正確率: 52.84%	校驗正確率: 47.78%
(tensor(87), 180)
訓練周期: 1 [640/1802 (43%)]	Total Loss: 0.72	training Loss: 0.70	val Loss: 0.70	訓練正確率: 53.72%	校驗正確率: 48.33%
(tensor(85), 180)
訓練周

(tensor(100), 180)
訓練周期: 13 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.80%	校驗正確率: 55.56%
(tensor(95), 180)
訓練周期: 13 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.77%	校驗正確率: 52.78%
(tensor(90), 180)
訓練周期: 14 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.78	val Loss: 0.69	訓練正確率: 46.88%	校驗正確率: 50.00%
(tensor(88), 180)
訓練周期: 14 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.15%	校驗正確率: 48.89%
(tensor(97), 180)
訓練周期: 14 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.81%	校驗正確率: 53.89%
(tensor(95), 180)
訓練周期: 14 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 47.58%	校驗正確率: 52.78%
(tensor(96), 180)
訓練周期: 14 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.17%	校驗正確率: 53.33%
(tensor(94), 180)
訓練周期: 15 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.75	val Loss: 0.69	訓練正確率: 40.62%	校驗正確率: 52.22%
(tensor(89)

(tensor(84), 180)
訓練周期: 27 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 47.38%	校驗正確率: 46.67%
(tensor(77), 180)
訓練周期: 27 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 48.09%	校驗正確率: 42.78%
(tensor(94), 180)
訓練周期: 28 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.75	val Loss: 0.69	訓練正確率: 53.12%	校驗正確率: 52.22%
(tensor(89), 180)
訓練周期: 28 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 51.42%	校驗正確率: 49.44%
(tensor(85), 180)
訓練周期: 28 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.81%	校驗正確率: 47.22%
(tensor(90), 180)
訓練周期: 28 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.70%	校驗正確率: 50.00%
(tensor(85), 180)
訓練周期: 28 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.62%	校驗正確率: 47.22%
(tensor(94), 180)
訓練周期: 29 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.66	val Loss: 0.69	訓練正確率: 50.00%	校驗正確率: 52.22%
(tensor(89),

(tensor(89), 180)
訓練周期: 41 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 44.89%	校驗正確率: 49.44%
(tensor(80), 180)
訓練周期: 41 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.71	訓練正確率: 46.13%	校驗正確率: 44.44%
(tensor(85), 180)
訓練周期: 41 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 46.57%	校驗正確率: 47.22%
(tensor(89), 180)
訓練周期: 41 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 47.41%	校驗正確率: 49.44%
(tensor(88), 180)
訓練周期: 42 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.79	val Loss: 0.69	訓練正確率: 40.62%	校驗正確率: 48.89%
(tensor(89), 180)
訓練周期: 42 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.71	val Loss: 0.70	訓練正確率: 46.31%	校驗正確率: 49.44%
(tensor(90), 180)
訓練周期: 42 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.36%	校驗正確率: 50.00%
(tensor(87), 180)
訓練周期: 42 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.29%	校驗正確率: 48.33%
(tensor(87

(tensor(85), 180)
訓練周期: 54 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.69	val Loss: 0.70	訓練正確率: 49.77%	校驗正確率: 47.22%
(tensor(76), 180)
訓練周期: 55 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.74	val Loss: 0.70	訓練正確率: 31.25%	校驗正確率: 42.22%
(tensor(84), 180)
訓練周期: 55 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 46.88%	校驗正確率: 46.67%
(tensor(90), 180)
訓練周期: 55 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.55%	校驗正確率: 50.00%
(tensor(89), 180)
訓練周期: 55 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 48.79%	校驗正確率: 49.44%
(tensor(89), 180)
訓練周期: 55 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 47.94%	校驗正確率: 49.44%
(tensor(80), 180)
訓練周期: 56 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.78	val Loss: 0.70	訓練正確率: 56.25%	校驗正確率: 44.44%
(tensor(89), 180)
訓練周期: 56 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.71	val Loss: 0.70	訓練正確率: 47.44%	校驗正確率: 49.44%
(tensor(99),

(tensor(87), 180)
訓練周期: 68 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 50.15%	校驗正確率: 48.33%
(tensor(85), 180)
訓練周期: 68 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.60%	校驗正確率: 47.22%
(tensor(89), 180)
訓練周期: 68 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.92%	校驗正確率: 49.44%
(tensor(96), 180)
訓練周期: 69 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.77	val Loss: 0.69	訓練正確率: 37.50%	校驗正確率: 53.33%
(tensor(87), 180)
訓練周期: 69 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.30%	校驗正確率: 48.33%
(tensor(93), 180)
訓練周期: 69 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.96%	校驗正確率: 51.67%
(tensor(102), 180)
訓練周期: 69 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.40%	校驗正確率: 56.67%
(tensor(80), 180)
訓練周期: 69 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.77%	校驗正確率: 44.44%
(tensor(

(tensor(92), 180)
訓練周期: 82 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.76	val Loss: 0.69	訓練正確率: 46.88%	校驗正確率: 51.11%
(tensor(80), 180)
訓練周期: 82 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 48.86%	校驗正確率: 44.44%
(tensor(82), 180)
訓練周期: 82 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 50.60%	校驗正確率: 45.56%
(tensor(82), 180)
訓練周期: 82 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 50.40%	校驗正確率: 45.56%
(tensor(100), 180)
訓練周期: 82 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.77%	校驗正確率: 55.56%
(tensor(100), 180)
訓練周期: 83 [0/1802 (0%)]	Total Loss: 0.71	training Loss: 0.69	val Loss: 0.69	訓練正確率: 53.12%	校驗正確率: 55.56%
(tensor(104), 180)
訓練周期: 83 [320/1802 (22%)]	Total Loss: 0.71	training Loss: 0.69	val Loss: 0.69	訓練正確率: 51.70%	校驗正確率: 57.78%
(tensor(88), 180)
訓練周期: 83 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.69	val Loss: 0.70	訓練正確率: 51.79%	校驗正確率: 48.89%
(tensor(92

(tensor(87), 180)
訓練周期: 95 [1280/1802 (87%)]	Total Loss: 0.70	training Loss: 0.69	val Loss: 0.70	訓練正確率: 51.83%	校驗正確率: 48.33%
(tensor(79), 180)
訓練周期: 96 [0/1802 (0%)]	Total Loss: 0.70	training Loss: 0.71	val Loss: 0.70	訓練正確率: 56.25%	校驗正確率: 43.89%
(tensor(91), 180)
訓練周期: 96 [320/1802 (22%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.70	訓練正確率: 47.44%	校驗正確率: 50.56%
(tensor(90), 180)
訓練周期: 96 [640/1802 (43%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.40%	校驗正確率: 50.00%
(tensor(88), 180)
訓練周期: 96 [960/1802 (65%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.89%	校驗正確率: 48.89%
(tensor(77), 180)
訓練周期: 96 [1280/1802 (87%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.70	訓練正確率: 48.86%	校驗正確率: 42.78%
(tensor(76), 180)
訓練周期: 97 [0/1802 (0%)]	Total Loss: 0.70	training Loss: 0.73	val Loss: 0.70	訓練正確率: 53.12%	校驗正確率: 42.22%
(tensor(84), 180)
訓練周期: 97 [320/1802 (22%)]	Total Loss: 0.70	training Loss: 0.70	val Loss: 0.70	訓練正確率: 49.15%	校驗正確率: 46.67%
(tensor(85),

#### label 2名稱

In [ ]:
#繪制訓練過程的誤差曲線，校驗集和測試集上的錯誤率。
plt.figure(figsize = (10, 7))
plt.plot(record, label = 'train_acc') #record記載了每一個打印周期記錄的訓練和校驗數據集上的準確度
plt.xlabel('Steps')
plt.ylabel('Error rate')
plt.legend()
plt.ylim(0, 1)

In [ ]:
records.append([np.mean(train_losses), np.mean(val_losses), train_r, val_r, losses])

In [ ]:
# 繪制誤差曲線
a = [i[0] for i in records]
b = [i[1] for i in records]
c = [i[2] for i in records]
d = [i[3] for i in records]
e = [i[4] for i in records]
plt.plot(a, label = 'Train Loss')
plt.plot(b, label = 'Valid Loss')
plt.plot(c, label = 'Train Accuracy')
plt.plot(d, label = 'Valid Accuracy')
plt.xlabel('Steps')
plt.ylabel('Loss & Accuracy')
plt.legend()

In [ ]:
torch.save(model,'bow.mdl')
model = torch.load('bow.mdl')

In [ ]:
#在測試集上分批運行，並計算總的正確率
vals = [] #記錄準確率所用列表

#對測試數據集進行循環
for data, target in zip(test_data, test_label):
    data, target = torch.tensor(data, dtype = torch.float).view(1,-1), torch.tensor(np.array([target]), dtype = torch.long)
    output = model(data) #將特征數據喂入網絡，得到分類的輸出
    val = rightness(output, target) #獲得正確樣本數以及總樣本數
    vals.append(val) #記錄結果

#計算準確率
rights = (sum([tup[0] for tup in vals]), sum([tup[1] for tup in vals]))
right_rate = 1.0 * rights[0].data.numpy() / rights[1]
right_rate